In [1]:
import pandas as pd
import ast
df_para =  pd.read_csv('data/kowshik_file_translated_para_phrased')
df_para['para_phrased'] = [ast.literal_eval(i) for i in df_para['para_phrased']]
df_title = pd.read_csv('data/videos_alphapolitica_tagged_translated.csv')
del df_title['Unnamed: 0']
del df_title['Unnamed: 0.1']
del df_title['len_keywords']
df_title['id'] = df_title['ids']
df_title['title'] = df_title['translated']
df_title = df_title[['id','title']]
df_one_shot = df_para.merge(df_title, on='id')
for j in range(5):
    df_one_shot['para_phrased_'+str(j)] = [i[j] for i in df_one_shot['para_phrased'].values]

In [3]:
from sentence_transformers import SentenceTransformer
sentences =df_one_shot['title']
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1')
embeddings = model.encode(sentences)

/Users/kowshik/projects/personal/alphapolitica/alphapenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
import sklearn 
similary = sklearn.metrics.pairwise.cosine_similarity(embeddings)
df_cosine=pd.DataFrame(similary)
df_cosine

In [35]:
from sklearn.cluster import AffinityPropagation
import numpy as np
X = np.array(embeddings)
clustering = AffinityPropagation(random_state=5).fit(X)
clustering

KeyboardInterrupt: 

In [34]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from tqdm import tqdm
cs = []
plt.figure(figsize=(10,6))
for i in tqdm(range(1, 11)):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(df_cosine)
    cs.append(kmeans.inertia_)
plt.plot(range(1, 11), cs)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('CS')
plt.show()

 20%|██        | 2/10 [05:43<22:54, 171.84s/it]


KeyboardInterrupt: 

<Figure size 1000x600 with 0 Axes>

In [19]:
[i for i in df_one_shot['translated'].values if 'murder' in i]

["Why did Naveen come to the fore in YS Viveka's murder case? Did CBI get important information from Naveen? What turn will the case take with Naveen Vijanna? Who is this Naveen based on Avinash's call data? Two new people have come to the fore. CBI questioned them both about the phone calls made. Who is Naveen who is so trustworthy? Naveen's family lives in Kapuram on Rajareddy Street in Pulivendula. They have been close to the YS family since the reign of Jagan Reddy's grandfather YS Rajareddy. In Pulivendula, Naveen is called Hariprasad. He was with them when he was in Hyderabad Lotus Pondy and now he has shifted to Tadepalli along with the family. Based on the call data, CBI has found that Veera spoke loyally to Bharti. It seems that CBI's aggressiveness is more pensionable. According to Naveen's inquiry, YS Bharti may also get notice. There are speculations in Kadapa district. After joining the CBI",
 "The CBI court has started the investigation process in the Viveka case. The mai

In [4]:
len(df_one_shot)

37493

In [43]:

df_para['para_phrased'].values[19]


['Several leaders of the YCP BC, who were from Gudivada rural mandal, joined the TDP in Krishna district. The former Sarpanch Basaviya leadership oversaw the recruitment of approximately 100 workers to the party. Jagan has only expressed his intention for the digulu March and is discussing pensions.',
 'In Krishna district, many leaders of YCP BC from Gudivada rural mandal joined the TDP under former Sarpanch Basaviya leadership. Around 100 workers also joined their ranks with Jagan who spoke about pensions without any form of pension.',
 'Several leaders of the YCP BC, who were from Gudivada rural mandal, joined the TDP in Krishna district. The former Sarpanch Basaviya leadership oversaw the recruitment of approximately 100 workers to the party. Jagan has only expressed his intention for the digulu March and is discussing pensions. Sakshi displayed a box of paper on the channel today.',
 "The YCP BC had many leaders from the Gudivada rural mandal who joined the TDP in Krishna district

In [51]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [50]:
sequence_to_classify = df_para['para_phrased'].values[19][2]
candidate_labels = ['farmers suicide', 'jagan mohan reddy', 'argiculture', 'TDP','telugu desam party', 'YCP']
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'Several leaders of the YCP BC, who were from Gudivada rural mandal, joined the TDP in Krishna district. The former Sarpanch Basaviya leadership oversaw the recruitment of approximately 100 workers to the party. Jagan has only expressed his intention for the digulu March and is discussing pensions. Sakshi displayed a box of paper on the channel today.',
 'labels': ['YCP',
  'TDP',
  'jagan mohan reddy',
  'telugu desam party',
  'argiculture',
  'farmers suicide'],
 'scores': [0.5806145071983337,
  0.26428067684173584,
  0.06777744740247726,
  0.05234777182340622,
  0.020734990015625954,
  0.014244611375033855]}

In [18]:
df_para['para_phrased'].values[0]

["There is a slight setback in the party due to Venkateta's state movement. It would be better for the leadership to take more responsibility as there is no visible response, and since most of the people who work for it are party workers, they should also consider taking action when problems arise.",
 'It would be better if the party tried to do more than just damage it, as there is no change in its affairs. The leadership must also think about this because those who work for the cause are all party workers. If problems arise during discussions like this, the parties will make a statement, but once again, they will say something.',
 "Although the party has suffered some harm from Venkateta's state movement, it is necessary for the leadership to take more responsibility in dealing with this issue.",
 "Although the party has suffered some harm from Venkateta's state movement, it is necessary for the leadership to take more responsibility in dealing with this issue. The best course of act

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cpu"

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [22]:
from tqdm import tqdm
para_phrased = []
for i in tqdm(df_trans['translated'].values):
    para_phrased.append(paraphrase(i))

  0%|          | 0/37554 [00:00<?, ?it/s]/Users/kowshik/projects/personal/alphapolitica/alphapenv/lib/python3.8/site-packages/transformers/generation/beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(
  1%|          | 246/37554 [13:23<33:52:06,  3.27s/it]


KeyboardInterrupt: 

2.086333333333333

In [7]:
df_trans['translated'].values[3]

"This is Karma for our state, yes or no, brother, if this happens, the shameless MLAs who killed Baba, at least did not speak, they are also criminals, brother, we need to remember that. I have given more than I have brought. Now this Chief Minister will complete it, don't you get angry with us? This is the karma of our state. Is it true or not? Today, those who give power to people like this have become extinct. Or is it the brother who is asking you to pay five thousand crores? If that worry is over, this worry is over"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import sys
from tqdm import tqdm
import pandas as pd

device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

start = int(sys.argv[1])
end = int(sys.argv[2])
print('start', sys.argv[1])
print('end', sys.argv[2])

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return res

df_trans = pd.read_csv('../data/kowshik_file_translated.csv')
del df_trans['Unnamed: 0']
del df_trans['sno']
para_phrased = []
for i in tqdm(df_trans['translated'].values):
    try:
        para_phrased.append(paraphrase(i))
    except:
        para_phrased.append(['None'])
df_trans['para_phrased'] = para_phrased
df_trans.to_csv('../data/kowshik_file_translated_para_phrased_' + str(start) +'_' +str(end) + '.csv')

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

sentences = ['This is an example sentence', 'Each sentence is converted']
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
print("Sentence embeddings:")
print(sentence_embeddings)


In [19]:
1100*5.5*2000 + 1500000

13600000.0

In [28]:
df = pd.read_csv('data/kowshik_file_translated_para_phrased_1_10.csv')

"['According to Keerthi Fandom, BRS is providing efficient governance in Telangana and focusing on the welfare of people from Rythu Bandha.', 'BRS is reportedly providing efficient governance in Telangana, according to Keerthi Fandom, who also stated that they are paying great attention to the welfare of people from Rythu Bandha.', 'The statement by Keerthi Fandom suggests that BRS is providing efficient governance in Telangana and paying a significant amount of attention to the welfare of people from Rythu Bandha.', 'Keerthi Fandom has stated that BRS is delivering efficient governance in Telangana and giving priority to the welfare of people from Rythu Bandha.', 'According to Keerthi Fandom, BRS is providing efficient governance in Telangana and focusing on the welfare of people from Rythu Bandha. People are getting good by drinking beer, exercising strength training, and consulting with Mr. KCR during the Bangalore program.']"

In [ ]:
import pandas as pd